In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv('F:/0Sem 7/ML Lab/amazon food review dataset/Reviews.csv')
df1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
score=df1.values[:,6]
text=df1.values[:,9]
reviews=np.vstack((score,text)).T
print(score.shape, text.shape, reviews.shape)

(568454,) (568454,) (568454, 2)


In [3]:
p=0
n=0
for i in range(reviews.shape[0]):
    if reviews[i,0] > 3:
        reviews[i,0]=0 #positive review
        p=p+1
    else:
        reviews[i,0]=1 #negative review
        n=n+1

In [4]:
reviews = reviews[reviews[:,0].argsort()]   #sort by 1st column

In [5]:
train=[]
for i in range(5000):
    train.append(reviews[i])
    
for i in range(443777,443777+5000):
    train.append(reviews[i])
    
train=np.asarray(train)
train.shape

(10000, 2)

In [6]:
my_reviews1=np.array([0,'This is a very good product. I am very happy with this item.'])
my_reviews2=np.array([1,'The product is very bad. I am very unsatisfied with the appearance.'])
my_reviews3=np.array([0,'It was one if the best items i have purchased. Very good.'])
my_reviews4=np.array([0,'All members of my family enjoyed the item. It is well thought.'])
my_reviews5=np.array([1,'Extremely poor quality. I hated the item and so did my brothers.'])

#train=np.vstack((train,my_reviews1))
#train=np.vstack((train,my_reviews2))
#train=np.vstack((train,my_reviews3))
#train=np.vstack((train,my_reviews4))
#train=np.vstack((train,my_reviews5))

In [7]:
import random as r
test=[]
for i in range(2000):
    index=r.randint(0,reviews.shape[0])
    test.append(reviews[index])
    
test=np.asarray(test)
test.shape

(2000, 2)

In [8]:
#test=np.vstack((test,my_reviews1))
#test=np.vstack((test,my_reviews2))
#test=np.vstack((test,my_reviews3))
#test=np.vstack((test,my_reviews4))
#test=np.vstack((test,my_reviews5))

In [9]:
train_all_words=[]
for i in range(train.shape[0]):
    train_all_words.append(train[i,1].split())
    
train_all_words = [item for sublist in train_all_words for item in sublist]

test_all_words=[]
for i in range(test.shape[0]):
    test_all_words.append(test[i,1].split())
    
test_all_words = [item for sublist in test_all_words for item in sublist]

from collections import Counter
def common_words(words, number_of_words, reverse=False):
    counter = Counter(words)
    return sorted(counter, key = counter.get, reverse=reverse)[:number_of_words]

train_least_common=common_words(train_all_words,200)
train_most_common=common_words(train_all_words,200,reverse=True)

test_least_common=common_words(test_all_words,200)
test_most_common=common_words(test_all_words,200,reverse=True)

In [10]:
for i in range(train.shape[0]):
    train[i,1]=train[i,1].split()
    
for i in range(test.shape[0]):
    test[i,1]=test[i,1].split()

In [11]:
for i in range(train.shape[0]):
    for item in train_most_common:
        if item in train[i,1]:
            train[i,1].remove(item)
    for item in train_least_common:
        if item in train[i,1]:
            train[i,1].remove(item)
            
for i in range(test.shape[0]):
    for item in test_most_common:
        if item in test[i,1]:
            test[i,1].remove(item)
    for item in test_least_common:
        if item in test[i,1]:
            test[i,1].remove(item)

In [12]:
for i in range(train.shape[0]):
    train[i,1]=" ".join(train[i,1])
    
for i in range(test.shape[0]):
    test[i,1]=" ".join(test[i,1])

In [13]:
train_docs=[]
test_docs=[]
train_label=[]
test_label=[]
for i in range(train.shape[0]):
    train_docs.append(train[i,1])
    train_label.append(train[i,0])
    
for i in range(test.shape[0]):
    test_docs.append(test[i,1])
    test_label.append(test[i,0])

In [14]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(train_docs)
vocab_size = len(t.word_index) + 1
#integer encode the documents
encoded_docs = t.texts_to_sequences(train_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)
#load the whole embedding into memory

embeddings_index = dict()
f = open('F:/0Sem 7/ML Lab/glove/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

#print('Loaded %s word vectors.' % len(embeddings_index))

#create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Using TensorFlow backend.


In [15]:
embedding_matrix.shape

(19814, 100)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#define model
model = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
model.add(layers.GRU(64, activation='relu', return_sequences=True))
model.add(layers.GRU(128, activation='relu', return_sequences=False))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1981400   
_________________________________________________________________
gru (GRU)                    (None, 100, 64)           31872     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               74496     
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,121,049
Trainable params: 139,649
Non-trainable params: 1,981,400
________________________________________

In [18]:
for layer in model.layers:
    print(layer.output_shape)

(None, 100, 100)
(None, 100, 64)
(None, 128)
(None, 128)
(None, 256)
(None, 1)


In [19]:
train_label=np.asarray(train_label)
train_label.shape

(10000,)

In [20]:
padded_docs.shape

(10000, 100)

In [21]:
# prepare tokenizer
t1 = Tokenizer()
t1.fit_on_texts(test_docs)
vocab_size1 = len(t1.word_index) + 1
#integer encode the documents
encoded_docs1 = t1.texts_to_sequences(test_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length1 = 100
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length1, padding='post')
test_label=np.asarray(test_label)
#print(padded_docs)

In [22]:
# fit the model
model.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=50, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
10000/10000 [==============================] - 102s 10ms/sample - loss: 0.6919 - accuracy: 0.5100 - val_loss: 0.7092 - val_accuracy: 0.7130
Epoch 2/50
10000/10000 [==============================] - 91s 9ms/sample - loss: 0.6886 - accuracy: 0.5231 - val_loss: 0.6941 - val_accuracy: 0.7370
Epoch 3/50
10000/10000 [==============================] - 97s 10ms/sample - loss: 0.6785 - accuracy: 0.5321 - val_loss: 0.8825 - val_accuracy: 0.7010
Epoch 4/50
10000/10000 [==============================] - 103s 10ms/sample - loss: 0.6631 - accuracy: 0.5506 - val_loss: 0.7053 - val_accuracy: 0.7305
Epoch 5/50
10000/10000 [==============================] - 103s 10ms/sample - loss: 0.6582 - accuracy: 0.5467 - val_loss: 0.7537 - val_accuracy: 0.7285
Epoch 6/50
10000/10000 [==============================] - 103s 10ms/sample - loss: 0.6372 - accuracy: 0.5608 - val_loss: 0.7749 - val_accuracy: 0.7215
Epoch 7/50
10000/10000 [========================

In [23]:
"Bidirectional Model"

'Bidirectional Model'

In [24]:
#define model
model1 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model1.add(e)
model1.add(layers.Bidirectional(layers.LSTM(64, activation='relu', return_sequences=True)))
model1.add(layers.Bidirectional(layers.LSTM(128, activation='relu', return_sequences=False)))
model1.add(layers.Flatten())
model1.add(layers.Dense(256,activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          84480     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 2,395,097
Trainable params: 413,697
Non-trainable params: 1,981,400
______________________________________

In [26]:
# fit the model
history1=model1.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=50, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
10000/10000 [==============================] - 114s 11ms/sample - loss: 0.6932 - accuracy: 0.4954 - val_loss: 0.6933 - val_accuracy: 0.2260
Epoch 2/50
10000/10000 [==============================] - 133s 13ms/sample - loss: 0.6932 - accuracy: 0.4902 - val_loss: 0.6938 - val_accuracy: 0.2260
Epoch 3/50
10000/10000 [==============================] - 134s 13ms/sample - loss: 0.6932 - accuracy: 0.4898 - val_loss: 0.6930 - val_accuracy: 0.7740
Epoch 4/50
10000/10000 [==============================] - 138s 14ms/sample - loss: 0.6932 - accuracy: 0.4958 - val_loss: 0.6922 - val_accuracy: 0.7740
Epoch 5/50
10000/10000 [==============================] - 139s 14ms/sample - loss: 0.6932 - accuracy: 0.4974 - val_loss: 0.6929 - val_accuracy: 0.7740
Epoch 6/50
10000/10000 [==============================] - 138s 14ms/sample - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6929 - val_accuracy: 0.7740
Epoch 7/50
10000/10000 [=====================